**Yahoo Finance Scrapping and scrapping cleaning** 

Here is where I will take the data from Yahoo Finance using selenium. 
The Jupyter is designed for the user to select which stock they want to analyze and then run the code. 
The code will run as follows:
    
    1. Import necessary libraries
    2. Declare variables for Options for the Selenium web scrapping.
    3. Import support files:
        a. df_companies = a csv containing ticker and respective company from the SEC (followed cleaning)
        b. df_other = a csv containing ticker and respective commodity, index or crypto
    4. User interaction:
        a. Request if Company or Other
        b. Select desired Company or Other
    5. Cleaning of the web scrapping - see details below

In [1]:
import requests
import pandas as pd
from time import sleep
import numpy as np
from datetime import datetime
import sidetable #bonus
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

import warnings
warnings.filterwarnings('ignore')


In [2]:
import sys
sys.path.append('../')

import src.functions_project as functions

In [3]:
opciones= Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
#hide as a robot
opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_argument('--start-maximized') #start maximized
opciones.add_argument('user.data-dir=selenium') #save cookies
opciones.add_argument('--incognito')#incognito window

In [4]:
df_companies = pd.read_csv('../data/companiesclean.csv')
df_other = pd.read_csv('../data/other_ticker.csv')

In [8]:
functions.scrapping_file()

Would you like to analyze a Company or Other(Index/Commodity/Crypto)?: other
        Type                         Other   ticker
0      Index  Dow Jones Industrial Average     ^DJI
1      Index                       S&P 500    ^GSPC
2      Index              NASDAQ Composite    ^IXIC
3      Index         CBOE Volatility Index     ^VIX
4  Commodity                          Gold     GC=F
5  Commodity                     Crude Oil     CL=F
6  Commodity                        Silver     SI=F
7     Crypto                   Bitcoin USD  BTC-USD
8     Crypto                   Etherun USD  ETH-USD
-------------------------------------------------------------------
Which of the following other?(copy/paste desired ticker): ^GSPC




====== WebDriver manager ======


----------------------------------
Let the scrapping begin for ^GSPC
----------------------------------


Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/99.0.4844.51/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\anton\.wdm\drivers\chromedriver\win32\99.0.4844.51]


**Scrapping** of Yahoo Finance:
    
    1. Declare the initial URL, home page of Yahoo Finance
    2. Declare an empty table 'tabla' where we will drop our data
    3. Accept cookies 
    4. IF formula based on the user answer, the user will either select Company or Other
        a. this selection will vary the element of searcing, for example if the user selects company the search will come from         'ticker' while on the other hand, if it comes from Other, then the selection will come from 'other'
    5. Get to the historical data page
    6. Select "MAX" in the calendar, which will allow to the whole table to expand. 
    7. Very important are the scrolling down. A while True loop has been given to scroll to the end of the page 17 times using        action chains
    8. Table is retreived and appended in the 'tabla' previously empty declared.
    9. This is then saven in da_tabla = pd.DataFrame(tabla)
                             da_tabla2 = da_tabla.copy()

**Cleaning of Data** Now that we have extracted our data, we need to clean it. Below are the following steps:
    
    1. Declare a new dataframe 'df' using loc to locate the first element of tabla and split by the page separator "\n"
        a. note only rows 6:-2 are taken because the other were informative of the website, not only the table, not needed.
    2. Create a combined date column which is df[9], a combination of the 3 date columns.
    3. Change column names to relevant and copying Yahoo Finance, while drop() on the ones used for d[9]
    4. Using lambda and replace to change all the commas to spaces, important for changing types.
    5. Change the format of 'date' to datetime
    6. UH OH we found a trouble, check below ! check.

**NOTE THIS CLEANING IS A FORMULA IN THE SRC CALLED defclean_scrapping()**

In [ ]:
da_tabla = pd.DataFrame(tabla)
da_tabla2 = da_tabla.copy()

In [ ]:
df = pd.DataFrame(da_tabla2.loc[0,0].split("\n")[6:-2])

In [ ]:
df.head(2)

In [ ]:
df_st = df[0].str.split(" ", expand = True)

In [ ]:
df_st.head(3)

In [ ]:
df_st[9] = df_st[0] + ' ' + df_st[1] + ' ' + df_st[2]

In [ ]:
df_st.head(2)

In [ ]:
df_st.columns = ['d1', 'd2', 'd3', 'Open','High','Low','Close','Adj Close','Volume','date']
df_st.drop(['d1','d2','d3'], axis = 1, inplace = True)

In [ ]:
df_st.head(2)

In [ ]:
df_st = df_st.apply(lambda x: x.str.replace(',',''))

In [ ]:
df_st.dtypes

In [ ]:
df_st['date'] = pd.to_datetime(df_st['date'], format='%b %d %Y')

In [ ]:
df_st.head(2)

In [ ]:
df_st.dtypes

In [ ]:
df_st.duplicated(['date']).sum()

In [ ]:
df_st.isnull().sum()

**When a stock pays dividend, that day is recorded twice in the historical data
but the values are 0, hence the 33 duplicates in the date. StockX has paid dividends 33times. 
Since I dont care about those days, because the day-data is there
I will create a new df called df_stocks, which will be the rows that are not null in the "close" column.** 

In [ ]:
df_stocks = df_st.loc[(~df_st['Close'].isnull())]

In [ ]:
df_stocks.head(2)

In [ ]:
#now let´s check if it took out those. 

df_stocks.duplicated(['date']).sum()

**Note** Okay better, but the numbers are impossible to use because they are strings, so will transform to float

In [ ]:
df_stocks['Open'] = df_stocks['Open'].astype(float)
df_stocks['High'] = df_stocks['High'].astype(float)
df_stocks['Low'] = df_stocks['Low'].astype(float)
df_stocks['Close'] = df_stocks['Close'].astype(float)
df_stocks['Adj Close'] = df_stocks['Adj Close'].astype(float)
df_stocks['Volume'] = df_stocks['Volume'].astype(float)

In [ ]:
df_stocks.dtypes

In [ ]:
df_stocks.head(2)

**NOW** I will bring the csv for the clean tweets and declare it as df_twt, then:
    
    1. Change the 'time' column of the df_twt into 'date' and into datetime format.
    2. Merge my previous df_stocks with df_twt using the date as unifier, creating df_atwt
    3. Double check the characteristics of the file
    4. Save new file for analysis
    

In [ ]:
df_twt = pd.read_csv('tweetsclean.csv')

In [ ]:
df_twt.tail(2)

In [ ]:
df_twt['date'] = pd.to_datetime(df_twt['fecha'])
df_twt.drop(['fecha'], axis = 1, inplace = True)

In [ ]:
df_twt.head(2)

In [ ]:
df_atwt = pd.merge(df_twt,df_stocks, how = 'outer', on = 'date')

In [ ]:
df_atwt.head(2)

In [ ]:
df_atwt.shape

In [ ]:
#si el mercado no está abierto, es fin de semana
df_atwt['market_op_cl'] = np.where(df_atwt['Open'] > 0, 'market_open', 'market_closed')

In [ ]:
df_atwt.duplicated().sum()

In [ ]:
df_atwt.stb.freq(['market_op_cl'])

In [ ]:
df_atwt.head(10)

In [ ]:
df_atwt.to_csv('analysis1.csv', index = False)